In [146]:
import os
import string
import math
from nltk.corpus import stopwords
stopWords = stopwords.words("english")

### First Step: preprocessing the data:
##### After preprocessing i save the preprocessed documents in the corresponding  new documents called preprocessed. for preprocessing I delete punctuations, covert each letter ro its lower case and delete stop words. (Just copied from the previous homeWorks)

In [147]:
folderPath = "F:/uni/Term7/IR/homeWorks/HW3/documents/document"
for fileName in os.listdir(folderPath):
    filePath = os.path.join(folderPath, fileName)
    if os.path.isfile(filePath):
        file = open(filePath, 'r', encoding='utf-8')
        content = file.read()
        translator = str.maketrans('', '', string.punctuation + "“”’—")
        contentPreprocessed = content.lower().translate(translator)
        terms = [word for word in list(contentPreprocessed.split()) if word not in stopWords]
        sortedTerms = sorted(terms)
        uniqueTerms = sorted(list(set(terms)))
        print(f"unique terms: {uniqueTerms}")
        with open(f"F:/uni/Term7/IR/homeWorks/HW3/preprocessed/{fileName[:-4]} preProcessed.txt", 'w', encoding='utf-8') as preProcessed:
            preProcessed.write(contentPreprocessed)


unique terms: ['45yearold', '830', 'added', 'along', 'already', 'another', 'around', 'avoid', 'awkwardly', 'bag', 'balance', 'bandage', 'benefit', 'better', 'bruised', 'businesses', 'cause', 'charities', 'charity', 'clear', 'community', 'continue', 'coupons', 'curb', 'curbs', 'day', 'declined', 'determined', 'didnt', 'discomfort', 'discussion', 'eager', 'elbow', 'enjoy', 'ensure', 'even', 'event', 'events', 'expressed', 'fell', 'filled', 'fine', 'finished', 'forward', 'fun', 'funds', 'good', 'goodie', 'gratitude', 'greenfield', 'hazard', 'however', 'hurt', 'incident', 'injured', 'insisting', 'johnson', 'keep', 'knee', 'landed', 'let', 'lisa', 'little', 'local', 'looking', 'lost', 'love', 'meanwhile', 'measures', 'medal', 'medical', 'month', 'need', 'next', 'noting', 'obstacles', 'offered', 'one', 'organizers', 'park', 'participants', 'participate', 'participating', 'path', 'pavement', 'perseverance', 'personnel', 'plans', 'proud', 'race', 'raised', 'received', 'resident', 'riverside', 

##### I also created the inverted index and posting list for calculating tf and idf later. (Just copied from the previous homeWorks)

In [148]:
dictionary = dict()
lAvg = 0
documentNumber = 1
folderPath = "F:/uni/Term7/IR/homeWorks/HW3/preprocessed"
for fileName in os.listdir(folderPath):
    filePath = os.path.join(folderPath, fileName)
    if os.path.isfile(filePath):
        file = open(filePath, 'r', encoding='utf-8')
        terms = list(file.read().split())
        lAvg += len(terms)/20
        uniqueTerms = sorted(list(set(terms)))
        for i in range(len(terms)):
            wordPositionDict = dict()
            wordPositionDict[documentNumber] = [i]
            if terms[i] in dictionary.keys():
                dictionary[terms[i]]["freq"] += 1
                if documentNumber not in dictionary[terms[i]]["postingList"].keys():
                    dictionary[terms[i]]["postingList"][documentNumber] = [i]
                else:
                    dictionary[terms[i]]["postingList"][documentNumber].append(i)     
            else:
                dictionary[terms[i]] = dict(freq = 1, postingList = wordPositionDict)
    documentNumber += 1
sortedDictionary  = {key: dictionary[key] for key in sorted(dictionary)}
sortedDictionary


{'1': {'freq': 1, 'postingList': {16: [300]}},
 '10': {'freq': 3, 'postingList': {12: [113], 16: [167], 17: [141]}},
 '100': {'freq': 1, 'postingList': {8: [205]}},
 '100000': {'freq': 1, 'postingList': {8: [129]}},
 '1015': {'freq': 1, 'postingList': {15: [103]}},
 '120': {'freq': 1, 'postingList': {10: [28]}},
 '1234': {'freq': 1, 'postingList': {15: [17]}},
 '15': {'freq': 2, 'postingList': {12: [115], 17: [143]}},
 '150': {'freq': 1, 'postingList': {3: [4]}},
 '15percent': {'freq': 1, 'postingList': {9: [38]}},
 '175000': {'freq': 1, 'postingList': {6: [452]}},
 '1999': {'freq': 1, 'postingList': {16: [290]}},
 '2': {'freq': 1, 'postingList': {18: [104]}},
 '20': {'freq': 2, 'postingList': {16: [169, 261]}},
 '200': {'freq': 1, 'postingList': {9: [149]}},
 '2000': {'freq': 1, 'postingList': {9: [64]}},
 '2010': {'freq': 1, 'postingList': {15: [46]}},
 '2015': {'freq': 1, 'postingList': {15: [21]}},
 '209': {'freq': 1, 'postingList': {16: [90]}},
 '214': {'freq': 1, 'postingList': {

##### I also created a list for storing all the queries in for easier accessing.

In [149]:
queryFile = open("./queries/queries/queries.txt", 'r')
queries = []
while True:
    query = queryFile.readline()
    if not query:
        break
    if query == '\n':
        continue
    translator = str.maketrans('', '', string.punctuation)
    queryList = list(query.lower().translate(translator).split())
    queries.append(queryList)

##### This is the corpus that contains all the terms. In this corpus similar terms could be repeated because we use this corpus to count frequency of a term in all documents i.e. $D_c$ 

In [150]:
corpus = []
folderPath = "F:/uni/Term7/IR/homeWorks/HW3/preprocessed"
for fileName in os.listdir(folderPath):
    filePath = os.path.join(folderPath, fileName)
    if os.path.isfile(filePath):
        file = open(filePath, 'r', encoding='utf-8')
        content = list(file.read().split())
        for i in content:
            corpus.append(i)
print(len(corpus))

12146


***

## Step 2: Implementing Okapi BM25:
##### in this step i have implemented the okapi NM25 as asked in the question. to do so, i used a helper function which does the computation part of the BM25 method and in the main function i get all the documents once and calculate the score of each document with the query based on the BM25 with basic weighting.
##### the reason we used default values for k1 and b i.e. k1 = 1.2 and b = 0.75 is that is realistic because users tend to write short queries and also documents have variable length and with these values we can handle short and long documents together.

In [151]:
def BM25helper(query, document, dictionary, lAvg):
    score = 0
    k1 = 1.2
    b = 0.75
    for t in query:
        if t in dictionary:
            df = math.log10(20/len(dictionary[t]["postingList"].keys()))    
            tf = document.count(t)
            makhraj = k1 * ((1 - b) + b * (len(document) / lAvg)) + tf
            score += df * ((k1 + 1) * tf) / makhraj
    return score

In [152]:
def BM25(query, k):
    folderPath = "F:/uni/Term7/IR/homeWorks/HW3/preprocessed"
    result = []
    finalResult = []
    for fileName in os.listdir(folderPath):
        filePath = os.path.join(folderPath, fileName)
        file = open(filePath, 'r', encoding='utf-8')
        document = list(file.read().split())
        result.append(BM25helper(query, document, sortedDictionary, lAvg))
    s = sorted(result.copy(), reverse=True)
    for i in range(k):
        finalResult.append(result.index(s[i])+1)
    return finalResult

#### Here is the result for each document. we can change 4 which is the second parameter of the function to get more relevant documents.

In [153]:
for query in queries:
    for i in query:
        print(i, end=" ")
    print()
    print(BM25(query, 4))

what safety measures should be implemented to prevent injuries during community sports events 
[5, 1, 7, 13]
how can local governments balance the economic benefits of community events with the potential disruptions they may cause to residents 
[7, 9, 4, 1]
what was the total weight of debris removed 
[10, 3, 16, 13]
who organized the community cleanup event 
[10, 3, 19, 8]
what items were found during the cleanup 
[10, 3, 19, 13]
what are the financial impacts of shortterm rentals 
[7, 9, 6, 14]
how do residents feel about rental regulations 
[7, 9, 16, 18]
what challenges arise from rising housing costs 
[17, 12, 16, 14]
how does housing instability affect mental health in communities 
[12, 19, 7, 4]
what resources are available for supporting mental health in neighborhoods 
[12, 19, 8, 4]
how can community connections mitigate challenges faced by residents 
[4, 2, 19, 6]
what consequences did the reckless driver face after the chase 
[20, 12, 13, 4]
how did charlie jenkins change af

***

## Step 3: Implementing the Language Model:
##### Here I have implemented the language model using Jelinek Mercer smoothing method with the default value lambda = 0.5. Like BM25 it uses a helper function for the main computation and in the main function we calculate this score for each document and return the k nearest documents to the query. To avoid runtime error in the calculation i used an if condition to skip terms with zero occurrence in the corpus because calculating the logarithm there could run into an error.
##### I used the Jelinek-Mercer smoothing method because it can handle the zeros efficiently and it's easy to implement and use. it is also more robust for query with different lengths. 

In [154]:
def languageModelHelper(query, document, corpus, lmda=0.5):
    score = 0
    for q in query:
        x = lmda*(corpus.count(q)/len(corpus)) + (1-lmda)*(document.count(q)/len(document))
        score += math.log10(x) if x != 0 else 0
    return score

In [155]:
def languageModel(query, k):
    result = []
    finalResult = []
    for fileName in os.listdir(folderPath):
        filePath = os.path.join(folderPath, fileName)
        file = open(filePath, 'r', encoding='utf-8')
        document = list(file.read().split())
        result.append(languageModelHelper(query, document, corpus))
    s = sorted(result.copy(), reverse=True)
    for i in range(k):
        finalResult.append(result.index(s[i])+1)
    return finalResult


In [156]:
for query in queries:
    for i in query:
        print(i, end=" ")
    print()
    print(languageModel(query, 4))

what safety measures should be implemented to prevent injuries during community sports events 
[5, 1, 7, 13]
how can local governments balance the economic benefits of community events with the potential disruptions they may cause to residents 
[9, 7, 4, 1]
what was the total weight of debris removed 
[10, 3, 16, 13]
who organized the community cleanup event 
[10, 19, 3, 8]
what items were found during the cleanup 
[10, 3, 19, 13]
what are the financial impacts of shortterm rentals 
[7, 9, 13, 18]
how do residents feel about rental regulations 
[7, 9, 16, 18]
what challenges arise from rising housing costs 
[17, 12, 16, 14]
how does housing instability affect mental health in communities 
[12, 7, 19, 4]
what resources are available for supporting mental health in neighborhoods 
[12, 19, 4, 8]
how can community connections mitigate challenges faced by residents 
[4, 2, 12, 6]
what consequences did the reckless driver face after the chase 
[20, 13, 12, 4]
how did charlie jenkins change a

***

## Step 4: Implementing the hybrid method
##### Firstly I implemented the vector space model using the previous homework. Then in the hybridModel function i have defined some weights called w1, w2 and w3 to normalize impact of each model in the final score. for getting the correct answers i have divided each document score to the maximum of the that model and also i have subtracted that value from the minimum. heres more explanation of why i have used this:
##### in the vector space model scores are integers and can be high up to even more than 100. In the BN25 model scores are floats and small and finally in the language model scores are negative and have high variance. all together for normalizing i used that formula to get number between 0 and 1 and also tried to make huge difference in the scores between documents. but still in the language model values are close to each other so i gave it more weight for more considering even small differences between scores.


In [157]:
def VSM(queryList):
    vectorSpaceModel = []
    for i in range(1, 21):
        vector = []
        for t in list(set(corpus)):
            if i in sortedDictionary[t]["postingList"].keys():
                vector.append(1)
            else:
                vector.append(0)
        vectorSpaceModel.append(vector)
    vector = []
    for t in list(set(corpus)):
        if t in queryList:
            vector.append(1)
        else:
            vector.append(0)
    similarityList = []
    for docNum in range(20):
        similarity = 0
        for i in range(len(list(set(corpus)))):
            similarity += vectorSpaceModel[docNum][i]*vector[i]
        similarityList.append(similarity)
    return similarityList

In [158]:
def hybridModel(query, k):
    vsm = VSM(query)   
    resultLM = []
    resultBM25 = []
    for fileName in os.listdir(folderPath):
        filePath = os.path.join(folderPath, fileName)
        file = open(filePath, 'r', encoding='utf-8')
        document = list(file.read().split())
        resultLM.append(languageModelHelper(query, document, corpus))
        resultBM25.append(BM25helper(query, document, dictionary, lAvg))
    maxVSM = max(vsm)
    maxLM = min(resultLM)
    maxBN25 = max(resultBM25)
    minVSM = min(vsm)
    minLM = max(resultLM)
    minBN25 = min(resultBM25)
    w1, w2, w3 = 0.5, 0.25, 0.25
    scores = []
    for i in range(20):
        scores.append(w1*((vsm[i]-minVSM)/(maxVSM-minVSM)) + w2*(abs((resultLM[i]-minLM)/(maxLM-minLM))) + w3*((resultBM25[i]-minBN25)/(maxBN25-minBN25)))
    finalResult = []    
    s = sorted(scores.copy(), reverse=True)
    for i in range(k):
        finalResult.append(scores.index(s[i])+1)
    return finalResult

In [159]:
for query in queries:
    for i in query:
        print(i, end=" ")
    print()
    print(hybridModel(query, 4))

what safety measures should be implemented to prevent injuries during community sports events 
[5, 13, 20, 7]
how can local governments balance the economic benefits of community events with the potential disruptions they may cause to residents 
[4, 8, 7, 5]
what was the total weight of debris removed 
[10, 3, 16, 18]
who organized the community cleanup event 
[10, 19, 3, 6]
what items were found during the cleanup 
[10, 19, 8, 6]
what are the financial impacts of shortterm rentals 
[7, 9, 6, 19]
how do residents feel about rental regulations 
[7, 16, 19, 17]
what challenges arise from rising housing costs 
[17, 12, 14, 6]
how does housing instability affect mental health in communities 
[12, 19, 14, 17]
what resources are available for supporting mental health in neighborhoods 
[19, 12, 8, 14]
how can community connections mitigate challenges faced by residents 
[6, 4, 19, 16]
what consequences did the reckless driver face after the chase 
[13, 12, 20, 17]
how did charlie jenkins chan

##### result in this step is more reliable because it consider all the three methods above and if a method is wrongly good for a document it reduces its impact.

***


##### Since the dataset is not judged at all we can't know which documents are relevant and which ones are not relevant, so recall means nothing in this situation, furthermore we can not compute 11-point interpolation since we can not compute recall for this dataset. in conclusion without further judgment we can not compute any metric for our models.
##### But here is my own comparison with checking documents with one-view judgment by myself: most of the results are the same specially about the first and second highest score retrieved documents, but in third and fourth one i think the hybrid method is doing better and return more relevant documents. comparing the result with the both methods in HW2 result are almost similar and but i think in the all differences in this homeworks results are better in quick looking on documents.
##### Finally we can't make a precise comparison between all the models but if we can say approximately the hybrid model is doing better than other models.

***